In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
from matplotlib.animation import FuncAnimation

# Set up the figure and 3D axis
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# Parameters for the animation
frames = 100  # Number of frames for the animation
car_speed = 0.1  # Speed of the car movement along x-axis

# Function to plot a rectangular prism (used for car body and wheels)
def plot_prism(ax, x, y, z, dx, dy, dz, color='b'):
    xx = [x, x+dx, x+dx, x, x, x+dx, x+dx, x]
    yy = [y, y, y+dy, y+dy, y, y, y+dy, y+dy]
    zz = [z, z, z, z, z+dz, z+dz, z+dz, z+dz]
    vertices = np.array([xx, yy, zz])
    edges = [[0,1], [1,2], [2,3], [3,0], [4,5], [5,6], [6,7], [7,4], [0,4], [1,5], [2,6], [3,7]]
    for edge in edges:
        ax.plot(vertices[0, edge], vertices[1, edge], vertices[2, edge], color=color)

# Function to plot a triangular mountain peak
def plot_mountain(ax, x_base, y_base, z_base, width, depth, height, color='gray'):
    x = [x_base, x_base + width, x_base + width/2]
    y = [y_base, y_base, y_base + depth]
    z = [z_base, z_base, z_base + height]
    ax.plot_trisurf(x, y, z, color=color, alpha=0.8)

# Initialize the scene
def init():
    ax.set_xlim(-10, 20)
    ax.set_ylim(-10, 10)
    ax.set_zlim(0, 10)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    # Plot the mountains in the background (at z = 0, y = -5)
    for x in range(-10, 15, 5):
        plot_mountain(ax, x, -8, 0, 4, 4, np.random.uniform(5, 8), color='gray')
    
    return []

# Update function for the animation
def update(frame):
    ax.cla()  # Clear the previous frame
    init()  # Redraw the mountains and set limits
    
    # Car position (moves along x-axis)
    car_x = -10 + frame * car_speed
    
    # Plot the car (body and wheels)
    # Car body (rectangular prism)
    plot_prism(ax, car_x, 0, 1, 4, 2, 1, color='blue')  # Main body
    plot_prism(ax, car_x + 1, 0, 2, 2, 2, 0.5, color='blue')  # Cabin
    
    # Wheels (smaller prisms)
    wheel_radius = 0.5
    plot_prism(ax, car_x, 0, 0.5, 0.5, 0.5, wheel_radius, color='black')  # Front left
    plot_prism(ax, car_x + 3.5, 0, 0.5, 0.5, 0.5, wheel_radius, color='black')  # Rear left
    plot_prism(ax, car_x, 1.5, 0.5, 0.5, 0.5, wheel_radius, color='black')  # Front right
    plot_prism(ax, car_x + 3.5, 1.5, 0.5, 0.5, 0.5, wheel_radius, color='black')  # Rear right
    
    return []

# Create the animation
ani = FuncAnimation(fig, update, frames=frames, init_func=init, blit=False)

# Save frames as images and compile into a video
frame_list = []
for i in range(frames):
    update(i)
    fig.canvas.draw()
    # Convert the plot to an image
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    frame_list.append(image)

# Use OpenCV to save the frames as a video
height, width, _ = frame_list[0].shape
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/kaggle/working/car_mountain_video.avi', fourcc, 20.0, (width, height))
for frame in frame_list:
    out.write(frame)
out.release()
plt.close()

print("Video saved as car_mountain_video.avi in /kaggle/working/")

Video saved as car_mountain_video.avi in /kaggle/working/
